In [0]:
#Importing the libraries
import pandas as pd       
import numpy as np
np.set_printoptions(suppress=True)
from google.colab import files
import io

In [0]:
#Uploading the dataset
data = files.upload()

Saving oasis_longitudinal.csv to oasis_longitudinal (1).csv


In [0]:
#Reading the dataset
df = pd.read_csv(io.BytesIO(data['oasis_longitudinal.csv']))

In [0]:
  #Dropping the unncessary columns
  SubjectID= df['Subject ID']
  df=df.drop(['Subject ID','MRI ID','Visit', 'Hand'],axis=1)

In [0]:
#Converting the Group column values to numeric values
df['Group']=df['Group'].replace(['Converted'],['Demented'])
df['Group']=df['Group'].replace(['Nondemented', 'Demented'], [0,1])

In [0]:
df.head(2)

,Group,MR Delay,M/F,Age,EDUC,SES,MMSE,CDR,eTIV,nWBV,ASF
0,0,0,M,87,14,2.0,27.0,0.0,1987,0.696,0.883
1,0,457,M,88,14,2.0,30.0,0.0,2004,0.681,0.876


In [0]:
df.describe()

,Group,MR Delay,Age,EDUC,SES,MMSE,CDR,eTIV,nWBV,ASF
count,373.000000,373.000000,373.000000,373.000000,354.000000,371.000000,373.000000,373.000000,373.000000,373.000000
mean,0.490617,595.104558,77.013405,14.597855,2.460452,27.342318,0.290885,1488.128686,0.729568,1.195461
std,0.500583,635.485118,7.640957,2.876339,1.134005,3.683244,0.374557,176.139286,0.037135,0.138092
min,0.000000,0.000000,60.000000,6.000000,1.000000,4.000000,0.000000,1106.000000,0.644000,0.876000
25%,0.000000,0.000000,71.000000,12.000000,2.000000,27.000000,0.000000,1357.000000,0.700000,1.099000
50%,0.000000,552.000000,77.000000,15.000000,2.000000,29.000000,0.000000,1470.000000,0.729000,1.194000
75%,1.000000,873.000000,82.000000,16.000000,3.000000,30.000000,0.500000,1597.000000,0.756000,1.293000
max,1.000000,2639.000000,98.000000,23.000000,5.000000,30.000000,2.000000,2004.000000,0.837000,1.587000


In [0]:
#Taking care of missing data
df["SES"].fillna(df.groupby("EDUC")["SES"].transform("median"), inplace=True) #For SES
df = df.dropna() #For MMSE
df.describe()

,Group,MR Delay,Age,EDUC,SES,MMSE,CDR,eTIV,nWBV,ASF
count,371.000000,371.000000,371.000000,371.000000,371.000000,371.000000,371.000000,371.000000,371.000000,371.000000
mean,0.487871,593.876011,77.018868,14.611860,2.477089,27.342318,0.287062,1489.876011,0.729526,1.193776
std,0.500528,636.636500,7.660864,2.877737,1.113256,3.683244,0.371912,174.990473,0.037229,0.136533
min,0.000000,0.000000,60.000000,6.000000,1.000000,4.000000,0.000000,1106.000000,0.644000,0.876000
25%,0.000000,0.000000,71.000000,12.000000,2.000000,27.000000,0.000000,1358.500000,0.700000,1.098500
50%,0.000000,552.000000,77.000000,15.000000,2.000000,29.000000,0.000000,1471.000000,0.729000,1.193000
75%,1.000000,869.500000,82.000000,16.000000,3.000000,30.000000,0.500000,1598.000000,0.756000,1.292000
max,1.000000,2639.000000,98.000000,23.000000,5.000000,30.000000,2.000000,2004.000000,0.837000,1.587000


In [0]:
#Taking care of discrete data
'''
df['age_bins'] = pd.cut(x=df['Age'], bins=[60, 69, 79, 89, 99], labels=[60, 70, 80, 90])
df['age_bins'].unique()'''

df['MMSE_bin'] = pd.cut(x=df['MMSE'], bins=[0, 13, 20, 24, 30], labels = ['12', '20', '24', '30'])
df['MMSE_bin'].unique()

[30, 24, 20, 12]
Categories (4, object): [12 < 20 < 24 < 30]

In [0]:
df.head(5)

,Group,MR Delay,M/F,Age,EDUC,SES,MMSE,CDR,eTIV,nWBV,ASF,MMSE_bin
0,0,0,M,87,14,2.0,27.0,0.0,1987,0.696,0.883,30
1,0,457,M,88,14,2.0,30.0,0.0,2004,0.681,0.876,30
2,1,0,M,75,12,3.0,23.0,0.5,1678,0.736,1.046,24
3,1,560,M,76,12,3.0,28.0,0.5,1738,0.713,1.010,30
4,1,1895,M,80,12,3.0,22.0,0.5,1698,0.701,1.034,24


In [0]:
#splitting Independent and dependent data.
y = df['Group'].values
X = df[['M/F', 'Age', 'EDUC', 'SES', 'eTIV', 'nWBV', 'ASF', 'MMSE_bin']].values

In [0]:
#Taking care of categorical data (Female/Male)
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
ct = ColumnTransformer([('encoder', OneHotEncoder(), [0])], remainder='passthrough')
X = np.array(ct.fit_transform(X), dtype=np.float)

In [0]:
ct = ColumnTransformer([('encoder', OneHotEncoder(), [8])], remainder='passthrough')
X = np.array(ct.fit_transform(X), dtype=np.float)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [0]:
from sklearn.model_selection import train_test_split #Splitting the train data to train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 0)

In [0]:
X_train

array([[   0.   ,    0.   ,    0.   , ..., 1453.   ,    0.721,    1.208],
       [   0.   ,    0.   ,    0.   , ..., 1215.   ,    0.71 ,    1.444],
       [   0.   ,    0.   ,    0.   , ..., 1460.   ,    0.695,    1.202],
       ...,
       [   0.   ,    0.   ,    0.   , ..., 1365.   ,    0.827,    1.285],
       [   0.   ,    0.   ,    0.   , ..., 1365.   ,    0.783,    1.286],
       [   0.   ,    1.   ,    0.   , ..., 1492.   ,    0.725,    1.176]])

In [0]:
# splitting into three sets
X_train_unscaled, X_test_unscaled, y_train, y_test = train_test_split(X, Y, test_size = 0.1, random_state=0)

# Feature scaling
scaler = MinMaxScaler().fit(X_train_unscaled)
X_train = scaler.transform(X_train_unscaled)
X_test = scaler.transform(X_test_unscaled)

In [0]:
np.any(np.isnan(X_train))

False

In [0]:
np.all(np.isfinite(X_train))

True

In [0]:
from sklearn.preprocessing import LabelEncoder
labelencoder_y1 = LabelEncoder()
y_train = labelencoder_y1.fit_transform(y_train)
# Fitting Logistic Regression to the Training set
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0)
classifier.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=0, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [0]:
# Predicting the Test set results
y_pred = classifier.predict(X_test)

In [0]:
from sklearn.model_selection import cross_val_score
cross_val_predict(classifier, X, y, cv=10)

NameError: ignored

In [0]:
#Calculating accuracy
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.868421052631579